# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [63]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json
import os

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csvfile = "output_data/cities.csv"
city_data = pd.read_csv(csvfile)
city_data

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Rikitea,-23.1203,-134.9692,22.17,73,100,3.69,PF,1663126466
1,1,Pouébo,-20.4000,164.5667,25.42,67,37,0.66,NC,1663126466
2,2,Port Blair,11.6667,92.7500,29.02,74,75,5.14,IN,1663126466
3,3,Kisi,9.0833,3.8500,21.27,99,77,1.46,NG,1663126468
4,4,Mataura,-46.1927,168.8643,10.27,72,90,2.33,NZ,1663126468
...,...,...,...,...,...,...,...,...,...,...
538,538,Mangan,27.5167,88.5333,27.80,77,90,2.45,IN,1663126733
539,539,Lebanon,36.2081,-86.2911,21.03,87,0,0.00,US,1663126733
540,540,Champasak,14.7500,106.0000,24.49,69,81,1.31,LA,1663126733
541,541,Khorramshahr,30.4397,48.1664,29.98,94,0,3.09,IR,1663126733


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store latitude and longitude in locations
locations = city_data[["Lat", "Lng"]]
humidity = city_data["Humidity"]

In [4]:
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [118]:
best_weather = city_data.drop(city_data.loc[city_data['Max Temp'] < 22].index, inplace=False)
best_weather = best_weather.drop(best_weather.loc[best_weather['Max Temp'] > 30].index, inplace=False)
best_weather = best_weather.drop(best_weather.loc[best_weather['Wind Speed'] > 4].index, inplace=False)
best_weather = best_weather.drop(best_weather.loc[best_weather['Humidity'] > 60].index, inplace=False)
hotel_df = best_weather.drop(best_weather.loc[best_weather['Cloudiness'] > 0].index, inplace=False)
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
18,18,Jalu,29.0331,21.5482,23.00,53,0,3.52,LY,1663126474
41,41,Awjilah,29.1081,21.2869,22.62,50,0,3.55,LY,1663126486
52,52,Alvorada,-12.4800,-49.1247,27.36,26,0,1.84,BR,1663126491
240,240,Ţahţā,26.7693,31.5021,25.63,38,0,2.83,EG,1663126583
271,271,Kumul,42.8000,93.4500,23.96,14,0,2.35,CN,1663126379
341,341,Chengde,40.9725,117.9361,29.43,29,0,3.41,CN,1663126636
363,363,Arak,34.0917,49.6892,23.31,18,0,1.80,IR,1663126646
367,367,Matamoros,25.5333,-103.2500,28.06,39,0,2.06,MX,1663126649
370,370,Aksu,41.1231,80.2644,25.35,22,0,1.45,CN,1663126650
414,414,Korla,41.7597,86.1469,24.63,20,0,1.10,CN,1663126672


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [119]:
hotel_df["Hotel Name"] = ""
hotel_df = hotel_df.iloc[: , 1:]

In [120]:
# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# set up a parameters dictionary and run request 
params = {
        "radius": 5000,
        "type": "lodging",
        "key": g_key
        }

for index, row in hotel_df.iterrows():
    lat = row['Lat']
    long = row['Lng']
    params['location'] = f'{lat},{long}'
    response = requests.get(base_url, params=params)
    hotels_data = response.json()
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels_data["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

In [121]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
18,Jalu,29.0331,21.5482,23.00,53,0,3.52,LY,1663126474,صاحبة الفخامة
41,Awjilah,29.1081,21.2869,22.62,50,0,3.55,LY,1663126486,المنزل
52,Alvorada,-12.4800,-49.1247,27.36,26,0,1.84,BR,1663126491,Cristal Plaza Hotel
240,Ţahţā,26.7693,31.5021,25.63,38,0,2.83,EG,1663126583,فندق الزيات
271,Kumul,42.8000,93.4500,23.96,14,0,2.35,CN,1663126379,Laoyutang Resort
341,Chengde,40.9725,117.9361,29.43,29,0,3.41,CN,1663126636,Super 8 Hotel
363,Arak,34.0917,49.6892,23.31,18,0,1.80,IR,1663126646,Zagros Hotel
367,Matamoros,25.5333,-103.2500,28.06,39,0,2.06,MX,1663126649,Quinta Belem
370,Aksu,41.1231,80.2644,25.35,22,0,1.45,CN,1663126650,Pudong Holiday Hotel
414,Korla,41.7597,86.1469,24.63,20,0,1.10,CN,1663126672,Gulisitan Hotel


In [134]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [136]:
# Add marker layer ontop of heat map
hotel_name = hotel_df["Hotel Name"].tolist()
cities = hotel_df["City"].tolist()
countries = hotel_df["Country"].tolist()
fig = gmaps.figure()
markers = gmaps.marker_layer(locations,
    info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure

fig

Figure(layout=FigureLayout(height='420px'))